In [1]:
!curl -o input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  1926k      0 --:--:-- --:--:-- --:--:-- 1948k


In [2]:
# The data is the tiny shakespeare dataset
with open('input.txt', 'r') as file:
    text = file.read()

In [3]:
print(f'Length of text: {len(text)}')

Length of text: 1115394


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# here are all the unique characters in the file
chars = sorted(list(set(text)))
voacb_size = len(chars)
print("".join(chars))
print(f'Vocab size: {voacb_size}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


In [6]:
# create a mapping from characters to indices and vice versa
char_to_index = {ch: i for i, ch in enumerate(chars)}
index_to_char = {i: ch for i, ch in enumerate(chars)}
encode = lambda x: [char_to_index[ch] for ch in x]
decode = lambda x: "".join([index_to_char[ch] for ch in x])

print(encode('hello there!'))
print(decode(encode('hello there!')))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 2]
hello there!


In [7]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(len(data) * 0.9)
train_data, val_data = data[:n], data[n:]

In [9]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context =x[:t+1]
    target = y[t]
    print(f"when input is {context} then target is {target}")

when input is tensor([18]) then target is 47
when input is tensor([18, 47]) then target is 56
when input is tensor([18, 47, 56]) then target is 57
when input is tensor([18, 47, 56, 57]) then target is 58
when input is tensor([18, 47, 56, 57, 58]) then target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) then target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4 # How many independent streams of data we want to process in parallel
block_size = 8 # The length of each stream of data

def get_batch(split):

    # Generate a small batch of inputs x and targets y

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i+1: i+ block_size + 1] for i in ix])

    return x, y

xb, yb = get_batch('train')
print(xb.shape, yb.shape)
print(xb)
print(yb)

print("---------------------------------")

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} then target is {target}")

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------------------------------
when input is [24] then target is 43
when input is [24, 43] then target is 58
when input is [24, 43, 58] then target is 5
when input is [24, 43, 58, 5] then target is 57
when input is [24, 43, 58, 5, 57] then target is 1
when input is [24, 43, 58, 5, 57, 1] then target is 46
when input is [24, 43, 58, 5, 57, 1, 46] then target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] then target is 39
when input is [44] then target is 53
when input is [44, 53] then target is 56
when input is [44, 53, 56] then target is 1
when input is [44, 53, 56, 1] then target is 58
when input is [44, 53,

In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, voacb_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table 
        # remember that the c or vocab_size are the number of characters in this case
        self.token_embedding = nn.Embedding(voacb_size, voacb_size)

    def forward(self, idx, targets = None):
        
        # idx and target are both (B, T) tensor of integers
        logits = self.token_embedding(idx) # (B, T, C)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) tensor of integers
        for _ in range(max_new_tokens):
            # Get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # becomes (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # becomes (B,1)
            # append the sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # becomes (B, T+1)
        return idx

    
m = BigramLanguageModel(voacb_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [23]:
# Create a pytorch optimizer    
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [26]:
batch_size = 32
for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.465181589126587


In [27]:
print(decode(m.generate(idx, 100)[0].tolist()))


He g sounomy.

TICLKINourushes t bladoonju cie st t tree IIOLENREThand t, aghath,
MI: Fol'storevend 
